In [1]:
import os

In [2]:
!gdalinfo --version

GDAL 2.3.2, released 2018/09/21


## Test image - Public Landsat 8 geotiff on Google Cloud

In [3]:
image_url = 'http://storage.googleapis.com/gcp-public-data-landsat/LC08/01/047/027/LC08_L1TP_047027_20130421_20170310_01_T1/LC08_L1TP_047027_20130421_20170310_01_T1_B4.TIF'

In [4]:
!gdalinfo /vsicurl/{image_url}

Driver: GTiff/GeoTIFF
Files: /vsicurl/http://storage.googleapis.com/gcp-public-data-landsat/LC08/01/047/027/LC08_L1TP_047027_20130421_20170310_01_T1/LC08_L1TP_047027_20130421_20170310_01_T1_B4.TIF
       /vsicurl/http://storage.googleapis.com/gcp-public-data-landsat/LC08/01/047/027/LC08_L1TP_047027_20130421_20170310_01_T1/LC08_L1TP_047027_20130421_20170310_01_T1_MTL.txt
Size is 7751, 7531
Coordinate System is:
PROJCS["WGS 84 / UTM zone 10N",
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",-123],
    PARAMETER["scale_factor",0.9996],
    PARAMETER["false_easting",5

# 0) direct download with curl - 1 GET request

In [5]:
%%time 

!curl -k -v -O {image_url}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0*   Trying 74.125.202.128...
* TCP_NODELAY set
* Connected to storage.googleapis.com (74.125.202.128) port 80 (#0)
> GET /gcp-public-data-landsat/LC08/01/047/027/LC08_L1TP_047027_20130421_20170310_01_T1/LC08_L1TP_047027_20130421_20170310_01_T1_B4.TIF HTTP/1.1
> Host: storage.googleapis.com
> User-Agent: curl/7.59.0
> Accept: */*
> 
< HTTP/1.1 200 OK
< X-GUploader-UploadID: AEnB2UqW43UqBSbdIMcknQdqVwt7YCivD7zn8jK9wAganu4QYojhQCHiqug__S2JrqqvvhPv6OqV5amXiRogzHPPVh5FUQohSIrvE7rbJqVvUisQurLsezc
< Expires: Tue, 18 Dec 2018 22:46:23 GMT
< Date: Tue, 18 Dec 2018 21:46:23 GMT
< Cache-Control: public, max-age=3600
< Last-Modified: Wed, 07 Jun 2017 01:42:26 GMT
< ETag: "c43a1fd0e598ce1837ff513ae9e0ef7a"
< x-goog-generation: 1496799746490963
< x-goog-metageneration

In [6]:
localfile = os.path.basename(image_url)
os.remove(localfile)

# 1) Pull single tile with gdal_translate - 3 GET requests

In [7]:
config = 'CPL_VSIL_CURL_USE_HEAD=NO GDAL_DISABLE_READDIR_ON_OPEN=EMPTY_DIR CPL_VSIL_CURL_ALLOWED_EXTENSIONS=.TIF'
config += ' CPL_CURL_VERBOSE=YES'
config += ' GDAL_HTTP_MERGE_CONSECUTIVE_RANGES=YES'
#cmd = f'{config} gdalmanage --debug ON copy /vsicurl/{image_url} {localfile}'
cmd = f'{config} gdal_translate --debug ON -srcwin 0 0 256 256 /vsicurl/{image_url} {localfile}'
print(cmd)

CPL_VSIL_CURL_USE_HEAD=NO GDAL_DISABLE_READDIR_ON_OPEN=EMPTY_DIR CPL_VSIL_CURL_ALLOWED_EXTENSIONS=.TIF CPL_CURL_VERBOSE=YES GDAL_HTTP_MERGE_CONSECUTIVE_RANGES=YES gdal_translate --debug ON -srcwin 0 0 256 256 /vsicurl/http://storage.googleapis.com/gcp-public-data-landsat/LC08/01/047/027/LC08_L1TP_047027_20130421_20170310_01_T1/LC08_L1TP_047027_20130421_20170310_01_T1_B4.TIF LC08_L1TP_047027_20130421_20170310_01_T1_B4.TIF


In [8]:
%%time

!{cmd}

GNM: GNMRegisterAllInternal
GNM: RegisterGNMFile
GNM: RegisterGNMdatabase
HTTP: libcurl/7.59.0 OpenSSL/1.0.2p zlib/1.2.11 libssh2/1.8.0
* Couldn't find host storage.googleapis.com in the .netrc file; using defaults
*   Trying 74.125.202.128...
* TCP_NODELAY set
* Connected to storage.googleapis.com (74.125.202.128) port 80 (#0)
> GET /gcp-public-data-landsat/LC08/01/047/027/LC08_L1TP_047027_20130421_20170310_01_T1/LC08_L1TP_047027_20130421_20170310_01_T1_B4.TIF HTTP/1.1
Host: storage.googleapis.com
Accept: */*

< HTTP/1.1 200 OK
< X-GUploader-UploadID: AEnB2Uruegypx7P2LLRvQDnn6DvTmbDwTwjpwP4Eo1Adut6fUyoNKtx1ADEoJoRsPuDwEZdOCs-z3NSSdGbGMJNuHo7nOgdj4VABpkRhFYZJWeZDl7KzvPU
< Expires: Tue, 18 Dec 2018 22:46:41 GMT
< Date: Tue, 18 Dec 2018 21:46:41 GMT
< Cache-Control: public, max-age=3600
< Last-Modified: Wed, 07 Jun 2017 01:42:26 GMT
< ETag: "c43a1fd0e598ce1837ff513ae9e0ef7a"
< x-goog-generation: 1496799746490963
< x-goog-metageneration: 1
< x-goog-stored-content-encoding: identity
< x-go

# 2) Pull row of tiles with gdal_translate - 3 GET requests b/c adjacent tile bytes are merged

In [ ]:
localfile = os.path.basename(image_url)
os.remove(localfile)

In [9]:
config = 'CPL_VSIL_CURL_USE_HEAD=NO GDAL_DISABLE_READDIR_ON_OPEN=EMPTY_DIR CPL_VSIL_CURL_ALLOWED_EXTENSIONS=.TIF'
config += ' CPL_CURL_VERBOSE=YES'
config += ' GDAL_HTTP_MERGE_CONSECUTIVE_RANGES=YES'
#cmd = f'{config} gdalmanage --debug ON copy /vsicurl/{image_url} {localfile}'
cmd = f'{config} gdal_translate --debug ON -srcwin 0 0 7751 256 /vsicurl/{image_url} {localfile}'
print(cmd)

CPL_VSIL_CURL_USE_HEAD=NO GDAL_DISABLE_READDIR_ON_OPEN=EMPTY_DIR CPL_VSIL_CURL_ALLOWED_EXTENSIONS=.TIF CPL_CURL_VERBOSE=YES GDAL_HTTP_MERGE_CONSECUTIVE_RANGES=YES gdal_translate --debug ON -srcwin 0 0 7751 256 /vsicurl/http://storage.googleapis.com/gcp-public-data-landsat/LC08/01/047/027/LC08_L1TP_047027_20130421_20170310_01_T1/LC08_L1TP_047027_20130421_20170310_01_T1_B4.TIF LC08_L1TP_047027_20130421_20170310_01_T1_B4.TIF


In [10]:
%%time

!{cmd}

GNM: GNMRegisterAllInternal
GNM: RegisterGNMFile
GNM: RegisterGNMdatabase
HTTP: libcurl/7.59.0 OpenSSL/1.0.2p zlib/1.2.11 libssh2/1.8.0
* Couldn't find host storage.googleapis.com in the .netrc file; using defaults
*   Trying 74.125.202.128...
* TCP_NODELAY set
* Connected to storage.googleapis.com (74.125.202.128) port 80 (#0)
> GET /gcp-public-data-landsat/LC08/01/047/027/LC08_L1TP_047027_20130421_20170310_01_T1/LC08_L1TP_047027_20130421_20170310_01_T1_B4.TIF HTTP/1.1
Host: storage.googleapis.com
Accept: */*

< HTTP/1.1 200 OK
< X-GUploader-UploadID: AEnB2UrQSvtTrGRx1BOKEQApwyUZMGfZX7mtijA1EVWuq_2403tDWfVR7mW3T9vOYE23W8RA7M8Pnbv8DtcLQKt5-wL7hS6eDxkHube0cKUFtCv5qhJjJno
< Expires: Tue, 18 Dec 2018 22:47:18 GMT
< Date: Tue, 18 Dec 2018 21:47:18 GMT
< Cache-Control: public, max-age=3600
< Last-Modified: Wed, 07 Jun 2017 01:42:26 GMT
< ETag: "c43a1fd0e598ce1837ff513ae9e0ef7a"
< x-goog-generation: 1496799746490963
< x-goog-metageneration: 1
< x-goog-stored-content-encoding: identity
< x-go

# 3) Pull entire file with gdal_translate - 17 GET requests b/c not all adjacent bytes merged

In [11]:
config = 'CPL_VSIL_CURL_USE_HEAD=NO GDAL_DISABLE_READDIR_ON_OPEN=EMPTY_DIR CPL_VSIL_CURL_ALLOWED_EXTENSIONS=.TIF'
config += ' CPL_CURL_VERBOSE=YES'
config += ' GDAL_HTTP_MERGE_CONSECUTIVE_RANGES=YES'
#cmd = f'{config} gdalmanage --debug ON copy /vsicurl/{image_url} {localfile}'
cmd = f'{config} gdal_translate --debug ON /vsicurl/{image_url} {localfile}'
print(cmd)

CPL_VSIL_CURL_USE_HEAD=NO GDAL_DISABLE_READDIR_ON_OPEN=EMPTY_DIR CPL_VSIL_CURL_ALLOWED_EXTENSIONS=.TIF CPL_CURL_VERBOSE=YES GDAL_HTTP_MERGE_CONSECUTIVE_RANGES=YES gdal_translate --debug ON /vsicurl/http://storage.googleapis.com/gcp-public-data-landsat/LC08/01/047/027/LC08_L1TP_047027_20130421_20170310_01_T1/LC08_L1TP_047027_20130421_20170310_01_T1_B4.TIF LC08_L1TP_047027_20130421_20170310_01_T1_B4.TIF


In [12]:
%%time

!{cmd} 2>&1 | tee

GNM: GNMRegisterAllInternal
GNM: RegisterGNMFile
GNM: RegisterGNMdatabase
HTTP: libcurl/7.59.0 OpenSSL/1.0.2p zlib/1.2.11 libssh2/1.8.0
* Couldn't find host storage.googleapis.com in the .netrc file; using defaults
*   Trying 74.125.202.128...
* TCP_NODELAY set
* Connected to storage.googleapis.com (74.125.202.128) port 80 (#0)
> GET /gcp-public-data-landsat/LC08/01/047/027/LC08_L1TP_047027_20130421_20170310_01_T1/LC08_L1TP_047027_20130421_20170310_01_T1_B4.TIF HTTP/1.1
Host: storage.googleapis.com
Accept: */*

< HTTP/1.1 200 OK
< X-GUploader-UploadID: AEnB2UrvFkAL2yq527UaHkgrfs-yWHCyGMacJu2X6ywExfVkkGRikN7w4u2V_qUN1CVWLnyCZ0JiqbqCFmWCRKfm1zYg7u4TX16sAwjKm-HHbcHZuemEcH0
< Expires: Tue, 18 Dec 2018 22:47:45 GMT
< Date: Tue, 18 Dec 2018 21:47:45 GMT
< Cache-Control: public, max-age=3600
< Last-Modified: Wed, 07 Jun 2017 01:42:26 GMT
< ETag: "c43a1fd0e598ce1837ff513ae9e0ef7a"
< x-goog-generation: 1496799746490963
< x-goog-metageneration: 1
< x-goog-stored-content-encoding: identity
< x-go